In [2]:

import cv2


In [ ]:
pip install opencv-python


In [ ]:
pip install keras

In [ ]:
pip install tensorflow

In [13]:
import cv2
import numpy as np
from cv2 import CascadeClassifier
import os
from cv2 import imread
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
import random
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator


def preprocessImages(raw_data_dir, preprocessed_data_dir):

  datagen = ImageDataGenerator(
      rotation_range=30,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      )

  # Define the target size for resizing
  target_size = (128, 128)

  # Define the number of augmented images to generate
  num_augmented_images = 100

  # Iterate over the raw images in the directory
  for image_file in os.listdir(raw_data_dir):
    image_path = os.path.join(raw_data_dir, image_file)

    print(image_file)
    for imageName in os.listdir(image_path):

      # Load the image
      image_dir = os.path.join(image_path, imageName)
      image = cv2.imread(image_dir)

      # Resize the image
      resized_image = cv2.resize(image, target_size)

      temp = 0
      input_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
      input_image = np.array(input_image)
      input_image = input_image.reshape((1,) + input_image.shape)  # Reshape for single image

      # Apply data augmentation to generate additional images
      for i in range(num_augmented_images):
        temp +=1
        if temp == num_augmented_images:
          break

        augmented_images = datagen.flow(input_image, batch_size=1, save_format='jpg')
        augmented_image = next(augmented_images)[0]
        augmented_image = augmented_image.astype(np.uint8)

          # Perform face detection and crop the image to the face region
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(augmented_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Iterate over the detected faces
        for (x, y, w, h) in faces:
            # store_dir = os.path.join(preprocessed_data_dir, image_file, imageName)
            # new_dir = os.path.join(store_dir, f'face_{image_file.split(".")[0]}')
            store_dir = os.path.join(preprocessed_data_dir, image_file)
            print(store_dir)
            os.makedirs(store_dir, exist_ok=True)
            # Crop the image to the face region
            cropped_image = augmented_image[y:y+h, x:x+w]

            # Save the augmented image with a unique filename
            augmented_image_filename = f'{imageName.split(".")[0]}_{i}.jpg'
            augmented_image_path = os.path.join(store_dir, augmented_image_filename)
            cv2.imwrite(augmented_image_path, cropped_image)

# Define the directory containing the raw images
# raw_data_dir = '/content/drive/MyDrive/Colab Notebooks/Assignment2/train'
raw_train_data_dir = 'D:\Documents\3rd year\AI\Assignment 2\myAssignment\datasets\train'
raw_validation_data_dir = 'D:\Documents\3rd year\AI\Assignment 2\myAssignment\datasets\validation'

# Define the output directory for the preprocessed dataset
preprocessed_train_data_dir = '/content/sample_data/preprocessed_dataset/train'
preprocessed_validation_data_dir = '/content/sample_data/preprocessed_dataset/validation'

# Create the output directory if it doesn't exist
os.makedirs(preprocessed_train_data_dir, exist_ok=True)
os.makedirs(preprocessed_validation_data_dir, exist_ok=True)

preprocessImages(raw_train_data_dir, preprocessed_train_data_dir)
preprocessImages(raw_validation_data_dir, preprocessed_validation_data_dir)







FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Documents/3rd year/AI/Assignment/datasets/train'

In [ ]:
# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''

# Specifying the folder where images are present



# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
# train_datagen = ImageDataGenerator(
#         shear_range=0.1,
#         zoom_range=0.1,
#         horizontal_flip=True,
#         rotation_range=20,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#          fill_mode='nearest')

# # Defining pre-processing transformations on raw images of testing data
# # No transformations are done on the testing images
TrainingImagePath = '/content/sample_data/preprocessed_dataset/train'
validationImagePath = '/content/sample_data/preprocessed_dataset/validation'

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# # Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=20,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        validationImagePath,
        target_size=(64, 64),
        batch_size=20,
        class_mode='categorical')

# Printing class labels for each face
training_set.class_indices

In [ ]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

In [ ]:
'''######################## Create CNN deep learning model ########################'''

from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(32, kernel_size=(3, 3), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

classifier.add(Convolution2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

'''# STEP--3 FLattening'''
classifier.add(Flatten())
classifier.add(Dropout(0.5))

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])


# Starting the model training
history = classifier.fit(
                    training_set,
                    epochs=7,
                    validation_data=test_set,
                    validation_steps=3)



In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
import numpy as np
import keras.utils as image

from IPython.display import Image
try:
  # filename = take_photo()
  # print('Saved to {}'.format(filename))

  # ImagePath='/content/' + filename
  ImagePath = '/content/sample_data/preprocessed_dataset/validation/face6/face6_1_1.jpg'
  test_image=image.load_img(ImagePath,target_size=(64, 64))
  test_image=image.img_to_array(test_image)

  test_image=np.expand_dims(test_image,axis=0)

  result=classifier.predict(test_image,verbose=0)
  # print(training_set.class_indices)

  print('####'*10)
  print('Prediction is: ',ResultMap[np.argmax(result)])

  # Show the image which was just taken.
  # display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))



In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
predicted

In [ ]:
 ImagePath = '/content/drive/MyDrive/AI/datasets/validation/face6/face6_1.jpg'
  test_image= cv2.imread(ImagePath)
  test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
  test_image = cv2.resize(test_image, (64, 64))
  # test_image=image.img_to_array(test_image)

  test_image=np.expand_dims(test_image,axis=0)

  result=classifier.predict(test_image)
  # print(training_set.class_indices)

In [ ]:
import cv2

# for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# resolution of the webcam
screen_width = 1280       # try 640 if code fails
screen_height = 720

# default webcam
stream = cv2.VideoCapture(0)

while(True):
    # capture frame-by-frame
    (grabbed, frame) = stream.read()
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # try to detect faces in the webcam
    faces = face_cascade.detectMultiScale(rgb, scaleFactor=1.3, minNeighbors=5)

    # for each faces found
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        color = (0, 255, 255) # in BGR
        stroke = 5
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, stroke)

    # show the frame
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):    # Press q to break out
        break                  # of the loop

# cleanup
stream.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


In [ ]:
import cv2

import numpy as np

from tensorflow import keras



# Load the pre-trained face detection model (e.g., Haar cascades or Dlib)

face_cascade = cv2.CascadeClassifier('path_to_face_cascade.xml')



# Load the pre-trained CNN model for face recognition

model = keras.models.load_model('path_to_trained_model.h5')



# Load a list of authorized individuals for attendance registration

authorized_individuals = ['John', 'Jane', 'Michael']  # Replace with your own list



# Function to perform face recognition on an input image

def recognize_face(image):

    # Convert the image to grayscale for face detection

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)



    # Perform face detection

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))



    for (x, y, w, h) in faces:

        # Extract the face region from the image

        face = gray[y:y+h, x:x+w]

        

        # Preprocess the face image (resize, normalize, etc.)

        face = cv2.resize(face, (64, 64))

        face = face / 255.0

        face = np.expand_dims(face, axis=0)

        

        # Perform face recognition by passing the face through the CNN model

        prediction = model.predict(face)

        

        # Retrieve the predicted label (person's identity)

        label_index = np.argmax(prediction)

        label = authorized_individuals[label_index]

        

        # Draw a rectangle and label around the detected face

        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    

    return image



# Capture video from the webcam

video_capture = cv2.VideoCapture(0)



while True:

    # Read a frame from the video stream

    ret, frame = video_capture.read()



    # Perform face recognition on the frame

    output = recognize_face(frame)



    # Display the output frame

    cv2.imshow('Face Recognition', output)



    # Break the loop if 'q' is pressed

    if cv2.waitKey(1) & 0xFF == ord('q'):

        break



# Release the video capture and close all windows

video_capture.release()

cv2.destroyAllWindows()

In [1]:
pip show tensorflow


Name: tensorflowNote: you may need to restart the kernel to use updated packages.

Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [ ]:
pip install tensorflow --upgrade
